In [1]:
from drn_interactions.interactions.loaders import BaseShockSlowInteractionsLoader
from drn_interactions.io import load_derived_generic
from drn_interactions.interactions.preprocessors import InteractionsPreprocessor
from drn_interactions.interactions.seqnmf import SeqNMF
import pandas as pd
from drn_interactions.config import Config, ExperimentInfo
from tqdm import tqdm

In [3]:
neuron_types = load_derived_generic("neuron_types.csv")
sessions = ExperimentInfo.foot_shock_sessions_10min
res = []

bin_widths = [0.05, 0.1, 0.5, 1]

loader_fac = lambda session_name, bin_width, block: BaseShockSlowInteractionsLoader(
    session_name=session_name, bin_width=bin_width, block=block
)
preprocessor_fac = lambda: InteractionsPreprocessor(
    z=False, minmax=True, gaussian_sigma=None,
)
model_fac = lambda K, L, Lambda: SeqNMF(K, L, Lambda)

for bin_width in tqdm(bin_widths):
    for block in ("pre", "base_shock"):
        for session in sessions:

            loader = loader_fac(session, bin_width, block=block)
            preprocessor = preprocessor_fac()
            model = model_fac(K=6, L=10, Lambda=0.1)
            spikes = preprocessor(loader())
            score = model.sequenciness_score(spikes.T)
            res.append(
                {
                    "session_name": session,
                    "k": 6,
                    "L": 10,
                    "bin_width": bin_width,
                    "score": score,
                    "block": block,
                }
            )

100%|██████████| 4/4 [20:40<00:00, 310.03s/it]


In [5]:
df_out = pd.DataFrame.from_records(res)

dd = Config.derived_data_dir / "seqnmf"
dd.mkdir(exist_ok=True)

df_out.to_csv(dd / "fs - sequenciness scores.csv")